In [9]:
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import os

# Stock health Dashboard

# Final batch process

In [3]:
from pathlib import Path

# Configuration
BASE_DIR = Path('/Users/andresuchitra/dev/missglam/autopo/notebook')
SUPPLIER_PATH = BASE_DIR / 'data/supplier.csv'
RAWPO_DIR = BASE_DIR / 'data/rawpo/csv'
INPUT_DIR = BASE_DIR / 'data/input'
RAWPO_XLSX_DIR = BASE_DIR / 'data/rawpo/xlsx'
STORE_CONTRIBUTION_PATH = BASE_DIR / 'data/store_contribution.csv'
OUTPUT_DIR = BASE_DIR / 'output/complete'
OUTPUT_EXCEL_DIR = BASE_DIR / 'output/excel'
OUTPUT_M2_DIR = BASE_DIR / 'output/m2'
OUTPUT_EMERGENCY_DIR = BASE_DIR / 'output/emergency'
TOP_100_SKU_DIR = BASE_DIR / 'data/top_100_sku'

NUMERIC_COLUMNS = [
    'HPP', 'Harga', 'Ranking', 'Grade', 'Terjual', 'Stok', 'Lost Days',
    'Velocity Capped', 'Daily Sales', 'Lead Time', 'Max. Daily Sales',
    'Max. Lead Time', 'Min. Order', 'Safety Stok', 'ROP', '3W Cover',
    'Sedang PO', 'Suggested', 'Amount', 'Promo Factor', 'Delay Factor',
    'Stock Cover', 'Days to Backup', 'Qty to Backup'
]

NA_VALUES = {
    'NAN', 'NA', '#N/A', 'NULL', 'NONE', '', '?', '-', 'INF', '-INF',
    '+INF', 'INFINITY', '-INFINITY', '1.#INF', '-1.#INF', '1.#QNAN'
}

def _patch_openpyxl_number_casting():
    """Ensure openpyxl won't crash when encountering NAN/INF in numeric cells."""
    print("Calling _patch_openpyxl_number_casting...")

    try:
        from openpyxl.worksheet import _reader

        original_cast = _reader._cast_number

        def _safe_cast_number(value):  # pragma: no cover - monkey patch
            if isinstance(value, str):
                if value.strip().upper() in NA_VALUES:
                    return 0
            try:
                return original_cast(value)
            except (ValueError, TypeError):
                return 0 if value in (None, '') else value

        _reader._cast_number = _safe_cast_number
    except Exception:
        # If patch fails we continue; runtime reader will still attempt default behaviour
        pass

def load_special_sku_60(path):
    print(f"Loading Special SKU with 60 days target cover data from {path}...")
    
    try:
        # Check file extension
        file_ext = str(path).lower().split('.')[-1]

        if file_ext == 'csv':
            # Read CSV with multiple possible delimiters and encodings
            try:
                df = pd.read_csv(path, sep=';', decimal=',', thousands='.', encoding='utf-8-sig')
            except (UnicodeDecodeError, pd.errors.ParserError):
                # Try with different encoding if UTF-8 fails
                df = pd.read_csv(path, sep=',', decimal='.', thousands=',', encoding='latin1')
                
        elif file_ext in ['xlsx', 'xls']:
            # Read Excel file
            df = pd.read_excel(path, engine='openpyxl')
        else:
            raise ValueError(f"Unsupported file format: {file_ext}. Please provide a CSV or Excel file.")
            
        # Basic data cleaning
        if not df.empty:
            # Strip whitespace from string columns
            df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
            
            # Convert column names to standard format
            df.columns = df.columns.str.strip()
            
            # Ensure SKU column is string type
            if 'SKU' in df.columns:
                df['SKU'] = df['SKU'].astype(str).str.strip()
                
        print(f"Successfully loaded Special SKU with 60 days target cover data with {len(df)} rows")
        
        return df
        
    except Exception as e:
        raise ValueError(f"Error loading Special SKU with 60 days target cover data from {path}: {str(e)}")

In [4]:
def load_top_100_sku_for_store(location, top_100_dir=TOP_100_SKU_DIR, expected_header_keys=None, max_rows=100):
    """Load Top 100 SKU data for a given store.

    This function will:
    - Find the matching Top 100 file for the store in ``top_100_dir``
    - Detect which row actually contains the header (can be on row 1, 2, 3, ...)
    - Read and return the cleaned DataFrame limited to top ``max_rows`` SKUs
    """
    try:
        location_upper = str(location).strip().upper()
        top_100_dir = Path(top_100_dir)

        if not top_100_dir.exists():
            print(f"Top 100 SKU directory does not exist: {top_100_dir}")
            return None

        # Use the same filename → store name logic to match files
        matching_files = []
        for path in sorted(top_100_dir.glob('*')):
            if not path.is_file():
                continue
            store_name = get_store_name_from_filename(path.name)
            if store_name == location_upper:
                matching_files.append(path)

        if not matching_files:
            print(f"No Top 100 SKU file found for store: {location_upper}")
            return None

        # If multiple files match, take the first one deterministically
        file_path = matching_files[0]
        print(f"Loading Top 100 SKU for {location_upper} from: {file_path}")

        suffix = file_path.suffix.lower()

        # Step 1: read raw file without assuming header row
        if suffix in ['.xlsx', '.xls']:
            raw_df = pd.read_excel(file_path, header=None, engine='openpyxl')
        else:
            read_ok = False
            raw_df = None
            for sep in [';', ',']:
                for enc in ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']:
                    try:
                        raw_df = pd.read_csv(file_path, header=None, sep=sep, encoding=enc)
                        read_ok = True
                        break
                    except Exception:
                        continue
                if read_ok:
                    break

            if not read_ok or raw_df is None:
                print(f"Failed to read Top 100 SKU file: {file_path}")
                return None

        if raw_df is None or raw_df.empty:
            print(f"Top 100 SKU file is empty: {file_path}")
            return None

                # Step 2: detect which row is the header
        # Strategy:
        #   1. Prefer any row containing a cell equal to "sku" (case-insensitive)
        #   2. Otherwise, take the first non-empty row
        header_row = None
        max_header_search = min(15, len(raw_df))  # look a bit deeper if needed

        for idx in range(max_header_search):
            row_values = raw_df.iloc[idx].astype(str).str.strip()
            lowered = [v.lower() for v in row_values]

            if "sku" in lowered:
                header_row = idx
                break

        if header_row is None:
            # fallback: first non-empty row
            for idx in range(max_header_search):
                if raw_df.iloc[idx].notna().any():
                    header_row = idx
                    break

        # As a final fallback, if still None but we have rows, use row 0
        if header_row is None:
            header_row = 0

        # Step 3: build df from raw_df using that header row
        header = raw_df.iloc[header_row].astype(str).str.strip().tolist()
        data = raw_df.iloc[header_row + 1 :].reset_index(drop=True)
        data.columns = header
        df = data

        # Basic cleaning
        df.columns = df.columns.astype(str).str.strip()

        # Normalize SKU column if present under any casing
        sku_col = None
        for c in df.columns:
            if str(c).strip().lower() == "sku":
                sku_col = c
                break

        if sku_col is not None:
            df[sku_col] = df[sku_col].astype(str).str.strip()
            # also expose as "SKU" for downstream code
            if sku_col != "SKU":
                df["SKU"] = df[sku_col]

        # Drop completely empty rows
        df = df.dropna(how='all')

        # Limit to top N rows
        if max_rows is not None and max_rows > 0:
            df = df.head(max_rows)

        print(f"Loaded Top 100 SKU for {location_upper}: {len(df)} rows")
        return df

    except Exception as e:
        print(f"Error loading Top 100 SKU for {location}: {str(e)}")
        return None

In [9]:
# Override merge_with_suppliers to prevent filling supplier data for rows with empty Brand
print("Applying empty-Brand-safe merge_with_suppliers override...")

def merge_with_suppliers(df_clean, supplier_df):
    """Merge PO data with supplier info, skipping fallback for blank Brand rows."""
    print("Merging with suppliers (override)...")

    supplier_clean = supplier_df.copy()
    supplier_clean['Nama Brand'] = supplier_clean['Nama Brand'].astype(str).str.strip()
    supplier_clean['Nama Store'] = supplier_clean['Nama Store'].astype(str).str.strip()
    supplier_clean = supplier_clean.drop_duplicates(subset=['Nama Brand', 'Nama Store'])

    df_clean = df_clean.copy()
    df_clean['Brand'] = df_clean['Brand'].astype(str).str.strip()
    df_clean['Toko'] = df_clean['Toko'].astype(str).str.strip()

    merged_df = pd.merge(
        df_clean,
        supplier_clean,
        left_on=['Brand', 'Toko'],
        right_on=['Nama Brand', 'Nama Store'],
        how='left',
        suffixes=('_clean', '_supplier')
    )

    merged_df['_brand_clean'] = merged_df['Brand'].astype(str).str.strip()
    unmatched_mask = merged_df['Nama Brand'].isna()
    fallback_mask = unmatched_mask & (merged_df['_brand_clean'] != '')

    if fallback_mask.any():
        print(
            f"Found {fallback_mask.sum()} rows without store match; attempting brand-only fallback for non-empty brands..."
        )
        unmatched_rows = merged_df[fallback_mask].copy()
        supplier_cols = [
            col for col in supplier_clean.columns if col in unmatched_rows.columns and col != 'Brand'
        ]
        unmatched_rows = unmatched_rows.drop(columns=supplier_cols, errors='ignore')
        unmatched_rows['Brand'] = unmatched_rows['_brand_clean']

        fallback_suppliers = supplier_clean.drop_duplicates(subset=['Nama Brand'])
        matched_fallback = pd.merge(
            unmatched_rows,
            fallback_suppliers,
            left_on='Brand',
            right_on='Nama Brand',
            how='left',
            suffixes=('_clean', '_supplier')
        )

        matched_initial = merged_df[~fallback_mask]
        merged_df = pd.concat([matched_initial, matched_fallback], ignore_index=True)

    merged_df = merged_df.drop(columns=['_brand_clean'], errors='ignore')

    supplier_columns = [
        'ID Supplier', 'Nama Supplier', 'ID Brand', 'ID Store',
        'Nama Store', 'Hari Order', 'Min. Purchase', 'Trading Term',
        'Promo Factor', 'Delay Factor'
    ]

    for col in supplier_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].fillna('' if merged_df[col].dtype == 'object' else 0)

    return merged_df

Applying empty-Brand-safe merge_with_suppliers override...


In [11]:
# Cell 1: Import libraries and setup
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from locale import atof
import numpy as np
from openpyxl.styles import numbers
from datetime import datetime

_patch_openpyxl_number_casting()

# Apply the formatting to numeric columns in your final output
def format_dataframe_display(df):
    # Make a copy to avoid SettingWithCopyWarning
    df_display = df.copy()
    
    # Apply formatting to numeric columns
    for col in df_display.select_dtypes(include=['int64', 'float64']).columns:
        df_display[col] = df_display[col].apply(
            lambda x: format_id_number(x, 2) if pd.notna(x) else x
        )
    
    return df_display

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_EXCEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_M2_DIR, exist_ok=True)
os.makedirs(OUTPUT_EMERGENCY_DIR, exist_ok=True)

# df_special_60 = load_special_sku_60(BASE_DIR / 'data/special_sku_60.csv')

# display(df_special_60)

def load_store_contribution(store_contribution_path):
    """Load and prepare store contribution data."""
    store_contrib = pd.read_csv(store_contribution_path, header=None, 
                              names=['store', 'contribution_pct'])
    # Convert store names to lowercase for case-insensitive matching
    store_contrib['store_lower'] = store_contrib['store'].str.lower()
    return store_contrib

def get_contribution_pct(location, store_contrib):
    """Get contribution percentage for a given location."""
    location_lower = location.lower()

    contrib_row = store_contrib[store_contrib['store_lower'] == location_lower]
    if not contrib_row.empty:
        return contrib_row['contribution_pct'].values[0]
    print(f"Warning: No contribution percentage found for {location}")

    return 100  # Default to 100% if not found

def load_supplier_data(supplier_path):
    """Load and clean supplier data."""
    print(f"Loading supplier data: {supplier_path}")
    df = pd.read_csv(supplier_path, sep=';', decimal=',').fillna('')
    df['Nama Brand'] = df['Nama Brand'].str.strip()
    return df

def merge_with_suppliers_v1(df_clean, supplier_df):
    """Merge PO data with supplier information."""
    print("Merging with suppliers...")
    
    # Clean supplier data
    supplier_clean = supplier_df.copy()
    supplier_clean['Nama Brand'] = supplier_clean['Nama Brand'].astype(str).str.strip()
    supplier_clean['Nama Store'] = supplier_clean['Nama Store'].astype(str).str.strip()
    
    # Deduplicate to prevent row explosion - Unique Brand+Store
    supplier_clean = supplier_clean.drop_duplicates(subset=['Nama Brand', 'Nama Store'])
    
    # Ensure PO data has clean columns for merging
    df_clean['Brand'] = df_clean['Brand'].astype(str).str.strip()
    df_clean['Toko'] = df_clean['Toko'].astype(str).str.strip()
    
    # 1. Primary Merge: Match on Brand AND Store (Toko)
    # This prioritizes the specific supplier for that store
    merged_df = pd.merge(
        df_clean,
        supplier_clean,
        left_on=['Brand', 'Toko'],
        right_on=['Nama Brand', 'Nama Store'],
        how='left',
        suffixes=('_clean', '_supplier')
    )
    
    # 2. Fallback: For unmatched rows, try to find ANY supplier for that Brand
    # Identify rows where merge failed (Nama Brand is NaN)
    unmatched_mask = merged_df['Nama Brand'].isna()
    
    if unmatched_mask.any():
        print(f"Found {unmatched_mask.sum()} rows without direct store match. Attempting fallback...")
        
        # Get the unmatched rows and drop the empty supplier columns
        unmatched_rows = merged_df[unmatched_mask].copy()
        supplier_cols = [col for col in supplier_clean.columns if col in unmatched_rows.columns and col != 'Brand']
        unmatched_rows = unmatched_rows.drop(columns=supplier_cols)
        
        # Create fallback supplier list (one per brand)
        # We take the first one found for each brand
        fallback_suppliers = supplier_clean.drop_duplicates(subset=['Nama Brand'])
        
        # Merge unmatched rows with fallback suppliers
        matched_fallback = pd.merge(
            unmatched_rows,
            fallback_suppliers,
            left_on='Brand',
            right_on='Nama Brand',
            how='left',
            suffixes=('_clean', '_supplier')
        )
        
        # Combine the initially matched rows with the fallback-matched rows
        matched_initial = merged_df[~unmatched_mask]
        merged_df = pd.concat([matched_initial, matched_fallback], ignore_index=True)
    
    # Clean up supplier columns
    supplier_columns = [
        'ID Supplier', 'Nama Supplier', 'ID Brand', 'ID Store', 
        'Nama Store', 'Hari Order', 'Min. Purchase', 'Trading Term',
        'Promo Factor', 'Delay Factor'
    ]
    for col in supplier_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].fillna('' if merged_df[col].dtype == 'object' else 0)
    
    return merged_df

def calculate_inventory_metrics(df_clean):
    """
    Calculate various inventory metrics including safety stock, reorder points, and PO quantities.
    
    Args:
        df_clean (pd.DataFrame): Input dataframe with required columns
        
    Returns:
        pd.DataFrame: Dataframe with added calculated columns
    """
    import numpy as np
    import pandas as pd
    
    # Ensure we're working with a copy to avoid SettingWithCopyWarning
    df = df_clean.copy()
    
    # Set display options
    pd.set_option('display.float_format', '{:.2f}'.format)
    # Normalise stock column name
    stock_col = 'Stok' if 'Stok' in df.columns else 'Stock'
    # Ensure is_top_100_sku exists (0 for non-top-100 by default)
    if 'is_top_100_sku' not in df.columns:
        df['is_top_100_sku'] = 0
    df['is_top_100_sku'] = pd.to_numeric(df['is_top_100_sku'], errors='coerce').fillna(0).astype(int)
    # Force the columns we need into numeric form
    numeric_cols = [
        stock_col, 'Daily Sales', 'Max. Daily Sales', 'Lead Time',
        'Max. Lead Time', 'Sedang PO', 'HPP', 'Harga', 'sales_contribution'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    try:
        # 1. Safety stock calculation
        df['Safety stock'] = (df['Max. Daily Sales'] * df['Max. Lead Time']) - (df['Daily Sales'] * df['Lead Time'])
        df['Safety stock'] = df['Safety stock'].apply(lambda x: np.ceil(x)).fillna(0).astype(int)
        
        # 2. Reorder point calculation
        df['Reorder point'] = np.ceil((df['Daily Sales'] * df['Lead Time']) + df['Safety stock']).fillna(0).astype(int)
        
        # 3. Stock cover for 30 or 60 days based on special SKUs
        # Default to 30 days for all SKUs
        df['target_days_cover'] = 30
        
        # 4. [DISABLED TO TOP 100 logic]Check if we have special SKUs and update their target days to 60
        # if df_special_60 is not None and not df_special_60.empty:
        #     # Find the SKU column in thxwe main dataframe (case-insensitive)
        #     sku_col = next((col for col in df.columns if col.lower() == 'sku'), None)
            
        #     # Find the SKU column in the special SKU dataframe (case-insensitive)
        #     special_sku_col = next((col for col in df_special_60.columns if col.lower() == 'sku'), None)
            
        #     if sku_col and special_sku_col:
        #         # Convert both to string and strip whitespace for matching
        #         df[sku_col] = df[sku_col].astype(str).str.strip()
        #         df_special_60[special_sku_col] = df_special_60[special_sku_col].astype(str).str.strip()
                
        #         # Update target_days_unit to 60 for special SKUs
        #         special_skus = set(df_special_60[special_sku_col].unique())
        #         df.loc[df[sku_col].isin(special_skus), 'target_days_cover'] = 60
        #     else:
        #         print("Warning: Could not find 'SKU' column in one of the dataframes")
        
        # Calculate target days cover based on the determined days
        df['qty_for_target_days_cover'] = (
            df['Daily Sales'] * df['target_days_cover']
        ).apply(lambda x: np.ceil(x)).fillna(0).astype(int)
        
        df['current_days_stock_cover'] = np.where(
            df['Daily Sales'] > 0,
            df[stock_col] / df['Daily Sales'],
            0
        )
        
        # 5. Is open PO flag
        df['is_open_po'] = np.where(
            (df['current_days_stock_cover'] < df['target_days_cover']) & 
            (df['Stok'] <= df['Reorder point']), 1, 0
        )
        
        # 6. Initial PO quantity
        df['initial_qty_po'] = df['qty_for_target_days_cover'] - df[stock_col] - df.get('Sedang PO', 0)
        df['initial_qty_po'] = (
            pd.Series(
                np.where(df['is_open_po'] == 1, df['initial_qty_po'], 0),
                index=df.index
            )
            .clip(lower=0)
            .astype(int)
        )
        
        # 7. Emergency PO quantity
        #    For non top-100 SKUs, this must always be 0
        base_emergency = np.where(
            df.get('Sedang PO', 0) > 0,
            np.maximum(0, (df['Max. Lead Time'] - df['current_days_stock_cover']) * df['Daily Sales']),
            np.ceil((df['Max. Lead Time'] - df['current_days_stock_cover']) * df['Daily Sales'])
        )
        df['emergency_po_qty'] = base_emergency * df['is_top_100_sku']
        # Clean up emergency PO quantities
        df['emergency_po_qty'] = (
            df['emergency_po_qty']
            .replace([np.inf, -np.inf], 0)
            .fillna(0)
            .clip(lower=0)
            .astype(int)
        )
        
        # 8. Updated regular PO quantity:
        # it will be same as initial suggested PO both for SKU Top 100 and others
        df['updated_regular_po_qty'] = df['initial_qty_po'].clip(lower=0).astype(int)
        
        # 9. Final updated regular PO quantity (enforce minimum order)
        df['final_updated_regular_po_qty'] = np.where(
            (df['updated_regular_po_qty'] > 0) & 
            (df['updated_regular_po_qty'] < df['Min. Order']),
            df['Min. Order'],
            df['updated_regular_po_qty']
        ).astype(int)
        
        # 10. Calculate costs if by multiplying with contribution percentage
        df['emergency_po_cost'] = (df['emergency_po_qty'] * df['HPP']).round(2)
        df['final_updated_regular_po_cost'] = (df['final_updated_regular_po_qty'] * df['HPP']).round(2)
        
        # Clean up any remaining NaN or infinite values
        df = df.fillna(0)
        
        return df
        
    except Exception as e:
        print(f"Error in calculate_inventory_metrics: {str(e)}")
        return df_clean

def clean_po_data(df, location, contribution_pct=100, padang_sales=None):
    """Clean and prepare PO data with contribution calculations."""
    try:
        # Create a copy to avoid modifying the original DataFrame
        df = df.copy()
        # remove duplicate SKU
        df = df.drop_duplicates(subset=['SKU'], keep='first')

        # Keep original column names but strip any extra whitespace
        df.columns = df.columns.str.strip()

        # Define required columns (using original case)
        required_columns = [
            'Brand', 'Kategori Brand', 'SKU', 'Nama', 'Toko', 'Stok',
            'Daily Sales', 'Max. Daily Sales', 'Lead Time',
            'Max. Lead Time', 'Min. Order', 'Sedang PO', 'HPP', 'Harga'
        ]
        
        # Find actual column names in the DataFrame (case-sensitive)
        available_columns = {col.strip(): col for col in df.columns}
        columns_to_keep = []
        
        for col in required_columns:
            if col in available_columns:
                columns_to_keep.append(available_columns[col])
            else:
                print(f"Warning: Column '{col}' not found in input data")
                # Add as empty column if it's required
                if col in ['Brand', 'SKU', 'HPP', 'Harga']:  # These are critical
                    df[col] = ''

        # Select only the columns we need
        df = df[[col for col in columns_to_keep if col in df.columns]]

        # Check for missing required columns
        missing_columns = [col for col in ['Brand', 'SKU', 'HPP', 'Harga'] if col not in df.columns]
        if missing_columns:
            raise ValueError(
                f"Missing required columns: {missing_columns}. "
                f"Available columns: {df.columns.tolist()}"
            )

        # Clean brand column
        if 'Brand' in df.columns:
            df['Brand'] = df['Brand'].astype(str).str.strip()

        # Convert SKU to string and clean it
        if 'SKU' in df.columns:
            df['SKU'] = df['SKU'].astype(str).str.strip()

        # Convert numeric columns with better error handling
        numeric_columns = [
            'Stok', 'Daily Sales', 'Max. Daily Sales', 'Lead Time',
            'Max. Lead Time', 'Sedang PO', 'HPP', 'Min. Order', 'Harga'
        ]

        for col in numeric_columns:
            if col in df.columns:
                try:
                    # First convert to string, clean, then to numeric
                    df[col] = (
                        df[col]
                        .astype(str)
                        .str.replace(r'[^\d.,-]', '', regex=True)  # Remove non-numeric except .,-
                        .str.replace(',', '.', regex=False)         # Convert commas to decimal points
                        .replace('', '0')                           # Empty strings to '0'
                        .astype(float)                              # Convert to float
                        .fillna(0)                                  # Fill any remaining NaNs with 0
                    )
                except Exception as e:
                    print(f"Warning: Could not convert column '{col}' to numeric: {str(e)}")
                    df[col] = 0  # Set to 0 if conversion fails

        # Add contribution percentage and calculate costs
        contribution_pct = float(contribution_pct)
        df['contribution_pct'] = contribution_pct

        # location_upper = location.upper()
        # exempt_stores = {"PADANG", "SOETA", "BALIKPAPAN"}
        # needs_padang_override = (location_upper not in exempt_stores) or (contribution_pct < 100)

        # Add 'Is in Padang' column
        # if padang_sales is not None:
        #     # Ensure padang_sales has the required columns
        #     padang_sales = padang_sales.copy()
        #     padang_sales.columns = padang_sales.columns.str.strip()
            
        #     # Convert SKU to string in both dataframes
        #     df['SKU'] = df['SKU'].astype(str).str.strip()
        #     padang_sales['SKU'] = padang_sales['SKU'].astype(str).str.strip()
            
        #     padang_skus = set(padang_sales['SKU'].unique())
        #     df['Is in Padang'] = df['SKU'].isin(padang_skus).astype(int)
        # else:
        #     print("Warning: No Padang sales data provided. 'Is in Padang' will be set to 0 for all SKUs.")
        #     df['Is in Padang'] = 0

        # if not needs_padang_override:
        #     return df

        # if padang_sales is None:
        #     raise ValueError(
        #         "Padang sales data is required for stores outside Padang/Soeta/Balikpapan "
        #         "or any store with contribution < 100%."
        #     )

        # Process Padang sales data
        # padang_df = padang_sales.copy()
        # padang_df.columns = padang_df.columns.str.strip()
        
        # Ensure required columns exist
        # required_cols = ['SKU', 'Daily Sales', 'Max. Daily Sales']
        # missing_cols = [col for col in required_cols if col not in padang_df.columns]
        # if missing_cols:
        #     raise ValueError(f"Missing required columns in Padang sales data: {missing_cols}")

        # Save original sales columns if they exist
        # if 'Daily Sales' in df.columns:
        #     df['Orig Daily Sales'] = df['Daily Sales']
        # if 'Max. Daily Sales' in df.columns:
        #     df['Orig Max. Daily Sales'] = df['Max. Daily Sales']

        # print("Overriding with Padang sales data...")
        
        # Ensure SKU is string in both dataframes before merge
        # df['SKU'] = df['SKU'].astype(str)
        # padang_df['SKU'] = padang_df['SKU'].astype(str)
        
        # DISABLE Padang sales override
        # Merge with Padang's sales data
        # df = df.merge(
        #     padang_df[['SKU', 'Daily Sales', 'Max. Daily Sales']].rename(columns={
        #         'Daily Sales': 'Padang Daily Sales',
        #         'Max. Daily Sales': 'Padang Max Daily Sales'
        #     }),
        #     on='SKU',
        #     how='left'
        # )
        
        # Calculate adjusted sales based on contribution and 'Is in Padang' flag
        # if 'Padang Daily Sales' in df.columns and 'Orig Daily Sales' in df.columns:
        #     df['Daily Sales'] = np.where(
        #         df['Is in Padang'] == 1,
        #         df['Padang Daily Sales'] * df['contribution_ratio'],
        #         df['Orig Daily Sales']
        #     )
            
        # if 'Padang Max Daily Sales' in df.columns and 'Orig Max. Daily Sales' in df.columns:
        #     df['Max. Daily Sales'] = np.where(
        #         df['Is in Padang'] == 1,
        #         df['Padang Max Daily Sales'] * df['contribution_ratio'],
        #         df['Orig Max. Daily Sales']
        #     )

        # Drop intermediate columns
        # columns_to_drop = [
        #     'Padang Daily Sales', 'Padang Max Daily Sales',
        # ]
        # df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

        # calculate sales contribution
        df['sales_contribution'] = df['Daily Sales'] * df['Harga']

        return df

    except Exception as e:
        print(f"Error in clean_po_data: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def get_store_name_from_filename(filename):
    """Extract store name from filename, handling different patterns."""
    # Remove file extension and split by spaces
    name_parts = Path(filename).stem.split()
    
    # Handle cases like "002 Miss Glam Pekanbaru.csv" -> "Pekanbaru"
    # or "01 Miss Glam Padang.csv" -> "Padang"
    if len(name_parts) >= 3 and name_parts[1].lower() == 'miss' and name_parts[2].lower() == 'glam':
        return ' '.join(name_parts[3:]).strip().upper()
    elif len(name_parts) >= 2 and name_parts[0].lower() == 'miss' and name_parts[1].lower() == 'glam':
        return ' '.join(name_parts[2:]).strip().upper()
    # Fallback: take everything after the first space
    elif ' ' in filename:
        return ' '.join(name_parts[1:]).strip().upper()
    return name_parts[0].upper()

def read_csv_file(file_path):
    # List of (separator, encoding) combinations to try
    formats_to_try = [
        (',', 'utf-8'),      # Standard CSV with comma
        (';', 'utf-8'),      # Semicolon with UTF-8
        (',', 'latin1'),     # Comma with Latin1
        (';', 'latin1'),     # Semicolon with Latin1
        (',', 'cp1252'),     # Windows-1252 encoding
        (';', 'cp1252')
    ]
    
    for sep, enc in formats_to_try:
        try:
            df = pd.read_csv(
                file_path,
                sep=sep,
                decimal=',',
                thousands='.',
                encoding=enc,
                engine='python'  # More consistent behavior with Python engine
            )
            # If we get here, the file was read successfully
            if not df.empty:
                return df
        except (UnicodeDecodeError, pd.errors.ParserError, pd.errors.EmptyDataError) as e:
            continue  # Try next format
        except Exception as e:
            print(f"Unexpected error reading {file_path} with sep='{sep}', encoding='{enc}': {str(e)}")
            continue
    
    # If we get here, all attempts failed
    print(f"Failed to read {file_path} with any known format")
    return None

def _find_sku_col(cols):
    for c in cols:
        if str(c).strip().lower() == "sku":
            return c
    return None

def process_po_file(file_path, supplier_df, store_contrib, df_padang, is_excel_folder=False):
    """Process a single PO file and return merged data and summary."""
    print(f"\nProcessing PO file: {file_path.name} ....")
    
    try:
        # Extract location from filename using the new function
        location = get_store_name_from_filename(file_path.name)
        print(f"  - Extracted location: {location}")  # Debug print
        
        contribution_pct = get_contribution_pct(location, store_contrib)
        
        # Read the CSV with error handling
        try:
            # Try reading with different encodings if needed
            if is_excel_folder:
                df = read_excel_file(file_path)
            else:
                df = read_csv_file(file_path)
            
            # Check if DataFrame is empty
            if df is None or df.empty:
                raise ValueError("File is empty")
                
            # Clean the data
            df_clean = clean_po_data(df, location, contribution_pct, df_padang)
            # Skip if cleaning failed
            if df_clean is None or df_clean.empty:
                raise ValueError("Data cleaning failed")

            # --- Top 100 SKU handling ---
            df_top_100 = load_top_100_sku_for_store(
                location,
                expected_header_keys=["SKU", "Nama", "Brand", "Toko"]
            )

            top_sku_col = _find_sku_col(df_top_100.columns) if df_top_100 is not None else None
            clean_sku_col = _find_sku_col(df_clean.columns)
            if (
                df_top_100 is not None
                and not df_top_100.empty
                and top_sku_col is not None
                and clean_sku_col is not None
            ):
                top_skus = set(df_top_100[top_sku_col].astype(str).str.strip())
                df_clean["is_top_100_sku"] = (
                    df_clean[clean_sku_col].astype(str).str.strip().isin(top_skus).astype(int)
                )
            else:
                print(f"  - No valid Top 100 data for {location}, setting is_top_100_sku = 0")
                df_clean["is_top_100_sku"] = 0

            if (
                df_top_100 is not None
                and not df_top_100.empty
                and "SKU" in df_top_100.columns
                and "SKU" in df_clean.columns
            ):
                top_skus = set(df_top_100["SKU"].astype(str).str.strip())
                df_clean["is_top_100_sku"] = (
                    df_clean["SKU"].astype(str).str.strip().isin(top_skus).astype(int)
                )
            else:
                print(f"  - No valid Top 100 data for {location}, setting is_top_100_sku = 0")
                df_clean["is_top_100_sku"] = 0
            # --- end Top 100 SKU handling ---
        
            # calculate metrics PO
            df_clean = calculate_inventory_metrics(df_clean)
            
            # Merge with suppliers
            merged_df = merge_with_suppliers(df_clean, supplier_df)
            # Generate summary
            padang_count = (merged_df['Nama Store'] == 'Miss Glam Padang').sum()
            other_supplier_count = ((merged_df['Nama Store'] != 'Miss Glam Padang') & 
                                  (merged_df['Nama Store'] != '')).sum()
            
            summary = {
                'file': file_path.name,
                'location': location,
                'contribution_pct': contribution_pct,
                'total_rows': len(merged_df),
                'padang_suppliers': int(padang_count),
                'other_suppliers': int(other_supplier_count),
                'no_supplier': int((merged_df['Nama Store'] == '').sum()),
                'status': 'Success'
            }
            
            return merged_df, summary
            
        except Exception as e:
            raise Exception(f"Error processing file data: {str(e)}")
            
    except Exception as e:
        error_msg = f"Error processing {file_path.name}: {str(e)}"
        print(f"  - {error_msg}")
        return None, {
            'file': file_path.name,
            'location': location if 'location' in locals() else 'Unknown',
            'contribution_pct': contribution_pct if 'contribution_pct' in locals() else 0,
            'total_rows': 0,
            'padang_suppliers': 0,
            'other_suppliers': 0,
            'no_supplier': 0,
            'status': f"Error: {str(e)[:100]}"  # Truncate long error messages
        }

def load_padang_data(padang_path):
    """Load Padang data from either CSV or Excel file.
    
    Args:
        padang_path: Path to the input file (CSV or XLSX)
        
    Returns:
        pd.DataFrame: Loaded and cleaned Padang data
        
    Raises:
        ValueError: If the file format is not supported or file cannot be read
    """
    print(f"Loading Padang data from {padang_path}...")
    
    # Check file extension
    file_ext = str(padang_path).lower().split('.')[-1]
    
    try:
        if file_ext == 'csv':
            # Read CSV with multiple possible delimiters and encodings
            try:
                df = pd.read_csv(padang_path, sep=';', decimal=',', thousands='.', encoding='utf-8-sig')
            except (UnicodeDecodeError, pd.errors.ParserError):
                # Try with different encoding if UTF-8 fails
                df = pd.read_csv(padang_path, sep=',', decimal='.', thousands=',', encoding='latin1')
                
        elif file_ext in ['xlsx', 'xls']:
            # Read Excel file
            df = pd.read_excel(padang_path, engine='openpyxl')
        else:
            raise ValueError(f"Unsupported file format: {file_ext}. Please provide a CSV or Excel file.")
            
        # Basic data cleaning
        if not df.empty:
            # Strip whitespace from string columns
            df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
            
            # Convert column names to standard format
            df.columns = df.columns.str.strip()
            
            # Ensure SKU column is string type
            if 'SKU' in df.columns:
                df['SKU'] = df['SKU'].astype(str).str.strip()
                
        print(f"Successfully loaded Padang data with {len(df)} rows")
        return df
        
    except Exception as e:
        raise ValueError(f"Error loading Padang data from {padang_path}: {str(e)}")

def format_number_for_csv(x):
    """Format numbers for CSV output with Indonesian locale (comma as decimal, dot as thousand)"""
    if pd.isna(x) or x == '':
        return x
    try:
        if isinstance(x, (int, float)):
            if x == int(x):  # Whole number
                return f"{int(x):,d}".replace(",", ".")
            else:  # Decimal number
                return f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
        return x
    except:
        return x

def clean_and_convert(df):
    """Clean and convert DataFrame columns to appropriate types."""
    if df is None or df.empty:
        return df

    # Make a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Convert all columns to string first to handle NaN/None consistently
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    # Define NA values that should be treated as empty/missing
    na_values = list(NA_VALUES)
    
    # Process each column
    for col in df.columns:
        # Replace NA values with empty string first (treating them as literals, not regex)
        df[col] = df[col].replace(na_values, '', regex=False)
        
        # Skip empty columns
        if df[col].empty:
            continue

        # Convert numeric columns
        if col in NUMERIC_COLUMNS:
            # Convert to numeric, coercing errors to NaN, then fill with 0
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        else:
            # For non-numeric columns, ensure they're strings and strip whitespace
            df[col] = df[col].astype(str).str.strip()
            # Replace empty strings with NaN and then fill with empty string
            # df[col] = df[col].replace('', np.nan).fillna('')
            # df[col] = df[col].replace('', np.nan).fillna('').infer_objects(copy=False)
            df[col] = df[col].replace('', np.nan).fillna('')
            df[col] = df[col].infer_objects(copy=False)

    return df

def read_excel_file(file_path):
    """
    Read an Excel file with robust error handling for problematic values.
    """
    try:
        print(f"\nReading excel file: {file_path.name}...")
        
        # First, read the file with openpyxl directly to handle the data more carefully
        from openpyxl import load_workbook
        
        # Load the workbook
        wb = load_workbook(
            filename=file_path,
            read_only=True,    # Read-only mode is faster and uses less memory
            data_only=True,    # Get the stored value instead of the formula
            keep_links=False   # Don't load external links
        )
        
        # Get the first sheet
        ws = wb.active
        
        # Get headers from the first row
        headers = []
        for idx, cell in enumerate(next(ws.iter_rows(values_only=True))):
            header = str(cell).strip() if cell not in (None, '') else f"Column_{idx + 1}"
            headers.append(header)
        
        # Initialize data rows
        data = []
        
        # Process each row
        for row in ws.iter_rows(min_row=2, values_only=True):  # Skip header row
            row_data = []
            for cell in row:
                if cell is None:
                    row_data.append('')
                    continue

                cell_str = str(cell).strip()
                if cell_str.upper() in NA_VALUES:
                    row_data.append('')
                else:
                    row_data.append(cell_str)
            
            # Only add row if it has data
            if any(cell != '' for cell in row_data):
                data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Normalize column data types
        df = clean_and_convert(df)
        
        print(f"✅ Successfully processed {file_path.name} with {len(df)} rows")
        return df
        
    except Exception as e:
        print(f"❌ Error processing {file_path.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def save_file(df, file_path, file_format='csv', **kwargs):
    """
    Save DataFrame to file with consistent extension and content type.
    
    Args:
        df: DataFrame to save
        file_path: Path object or string for the output file
        file_format: 'csv' or 'xlsx'
        **kwargs: Additional arguments to pass to to_csv or to_excel
        
    Returns:
        Path: The path where the file was saved
    """
    # Ensure file_path is a Path object
    file_path = Path(file_path)
    
    # Ensure the directory exists
    file_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Ensure the correct file extension
    if not file_path.suffix.lower() == f'.{file_format}':
        file_path = file_path.with_suffix(f'.{file_format}')
    
    # Make a copy to avoid modifying the original
    df_output = df.copy()
    
    # Common preprocessing
    if 'SKU' in df_output.columns:
        df_output['SKU'] = df_output['SKU'].astype(str).str.strip()
        if file_format == 'xlsx':
            # For Excel, wrap SKU in ="..." to preserve leading zeros
            df_output['SKU'] = df_output['SKU'].apply(lambda x: f'="{x}"')
    
    # Format numbers for CSV if needed
    if file_format == 'csv':
        numeric_cols = df_output.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            df_output[col] = df_output[col].apply(format_number_for_csv)
    
    # Save based on format
    if file_format == 'csv':
        df_output.to_csv(
            file_path, 
            index=False, 
            sep=';', 
            decimal=',', 
            encoding='utf-8-sig',
            **kwargs
        )
    elif file_format == 'xlsx':
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            df_output.to_excel(writer, index=False, **kwargs)
            
            # Format SKU column as text in Excel
            if 'SKU' in df_output.columns:
                ws = writer.sheets[list(writer.sheets.keys())[0]]
                sku_col_idx = df_output.columns.get_loc("SKU") + 1
                for row in ws.iter_rows(
                    min_row=2,  # Skip header
                    max_row=ws.max_row,
                    min_col=sku_col_idx,
                    max_col=sku_col_idx
                ):
                    for cell in row:
                        cell.number_format = numbers.FORMAT_TEXT
    else:
        raise ValueError(f"Unsupported file format: {file_format}")
    
    print(f"File saved to {file_path}")
    return file_path

def save_to_complete_format(df, filename, file_format='csv', **kwargs):
    """
    Save Complete format file with consistent extension.
    
    Args:
        df: Input DataFrame
        filename: Output filename (with or without extension)
        file_format: 'csv' or 'xlsx'
        **kwargs: Additional arguments for save_file
    """
    
    # Ensure output directory exists
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    # Save with consistent extension
    output_path = OUTPUT_DIR / filename

    return save_file(df, output_path, file_format=file_format, **kwargs)

def save_to_m2_format(df, filename, file_format='csv', **kwargs):
    """
    Save M2 format file with consistent extension.
    
    Args:
        df: Input DataFrame
        filename: Output filename (with or without extension)
        file_format: 'csv' or 'xlsx'
        **kwargs: Additional arguments for save_file
    """
    # Filter to only include rows with regular PO qty > 0
    df_filtered = df[df['final_updated_regular_po_qty'] > 0].copy()
    df_output = df_filtered[['Toko', 'SKU', 'HPP', 'final_updated_regular_po_qty']]
    
    # Ensure output directory exists
    OUTPUT_M2_DIR.mkdir(parents=True, exist_ok=True)
    
    # Save with consistent extension
    output_path = OUTPUT_M2_DIR / filename
    return save_file(df_output, output_path, file_format=file_format, **kwargs)

def save_to_emergency_po_format(df, filename, file_format='csv', **kwargs):
    """
    Save emergency PO format file with consistent extension.
    
    Args:
        df: Input DataFrame
        filename: Output filename (with or without extension)
        file_format: 'csv' or 'xlsx'
        **kwargs: Additional arguments for save_file
    """
    # Filter to only include rows with emergency PO qty > 0
    df_filtered = df[df['emergency_po_qty'] > 0].copy()
    df_output = df_filtered[[
        'Brand', 'SKU', 'Nama', 'Toko', 'HPP', 
        'emergency_po_qty', 'emergency_po_cost'
    ]]

    # Ensure output directory exists
    OUTPUT_EMERGENCY_DIR.mkdir(parents=True, exist_ok=True)
    
    # Save with consistent extension
    output_path = OUTPUT_EMERGENCY_DIR / filename
    return save_file(df_output, output_path, file_format=file_format, **kwargs)

def main():
    # Load data
    supplier_df = load_supplier_data(SUPPLIER_PATH)
    store_contrib = load_store_contribution(STORE_CONTRIBUTION_PATH)
    all_summaries = []

    current_date = datetime.now().strftime('%Y%m%d')
    CURRENT_DIR = INPUT_DIR / current_date

    # get padang df first
    df_padang = load_padang_data(CURRENT_DIR / '1. Miss Glam Padang.xlsx')

    # test_xlsx_convert()

    # Process each PO file
    for file_path in sorted(CURRENT_DIR.glob('*.xlsx')):
        try:
            merged_df, summary = process_po_file(file_path, supplier_df, store_contrib, df_padang, is_excel_folder=True)

            save_to_complete_format(merged_df, file_path.name, file_format='xlsx')
            save_to_complete_format(merged_df, file_path.name)
            save_to_m2_format(merged_df, file_path.name)
            save_to_emergency_po_format(merged_df, file_path.name)

            # summary['output_path'] = str(output_path)
            output_path = OUTPUT_DIR / file_path.name
            summary['output_path'] = str(output_path)

            
            # Print progress
            print(f"  - Location: {summary['location']}")
            print(f"  - Contribution: {summary['contribution_pct']}%")
            print(f"  - Rows processed: {summary['total_rows']}")
            print(f"  - 'Miss Glam Padang' suppliers: {summary['padang_suppliers']} rows")
            print(f"  - Other suppliers: {summary['other_suppliers']} rows")
            print(f"  - No supplier data: {summary['no_supplier']} rows")
            print(f"  - Saved to: {output_path}")
            
            all_summaries.append(summary)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {str(e)}")
            continue
    
    # Display final summary
    if all_summaries:
        print("\nProcessing complete! Summary:")
        summary_df = pd.DataFrame(all_summaries)
        display(summary_df)
        
        # Show sample of last processed file
        print("\nSample of the last processed file:")
        display(merged_df)
    else:
        print("\nNo files were processed successfully.")

# Run the main function
if __name__ == "__main__":
    main()


Calling _patch_openpyxl_number_casting...
Loading supplier data: /Users/andresuchitra/dev/missglam/autopo/notebook/data/supplier.csv
Loading Padang data from /Users/andresuchitra/dev/missglam/autopo/notebook/data/input/20251213/1. Miss Glam Padang.xlsx...
Successfully loaded Padang data with 37318 rows

Processing PO file: 1. Miss Glam Padang.xlsx ....
  - Extracted location: PADANG

Reading excel file: 1. Miss Glam Padang.xlsx...


/var/folders/8t/7219xcjd2dj829bf02_x9zy80000gn/T/ipykernel_3470/680799811.py:695: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('', np.nan).fillna('')


✅ Successfully processed 1. Miss Glam Padang.xlsx with 37318 rows
Loading Top 100 SKU for PADANG from: /Users/andresuchitra/dev/missglam/autopo/notebook/data/top_100_sku/1. Miss Glam Padang.xlsx
Loaded Top 100 SKU for PADANG: 100 rows
Merging with suppliers...
Found 26171 rows without direct store match. Attempting fallback...


Exception ignored in: <function ZipFile.__del__ at 0x10705de40>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/zipfile/__init__.py", line 1980, in __del__
    self.close()
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/zipfile/__init__.py", line 1997, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/1. Miss Glam Padang.xlsx
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/1. Miss Glam Padang.csv
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/m2/1. Miss Glam Padang.csv
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/emergency/1. Miss Glam Padang.csv
  - Location: PADANG
  - Contribution: 100%
  - Rows processed: 37318
  - 'Miss Glam Padang' suppliers: 11147 rows
  - Other suppliers: 537 rows
  - No supplier data: 25634 rows
  - Saved to: /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/1. Miss Glam Padang.xlsx

Processing PO file: 10. Miss Glam Palembang.xlsx ....
  - Extracted location: PALEMBANG

Reading excel file: 10. Miss Glam Palembang.xlsx...


...

/var/folders/8t/7219xcjd2dj829bf02_x9zy80000gn/T/ipykernel_3470/680799811.py:695: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('', np.nan).fillna('')


✅ Successfully processed 9. Miss Glam Medan.xlsx with 37161 rows
Loading Top 100 SKU for MEDAN from: /Users/andresuchitra/dev/missglam/autopo/notebook/data/top_100_sku/9. Miss Glam Medan.xlsx
Loaded Top 100 SKU for MEDAN: 100 rows
Merging with suppliers...
Found 26431 rows without direct store match. Attempting fallback...
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/9. Miss Glam Medan.xlsx
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/9. Miss Glam Medan.csv
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/m2/9. Miss Glam Medan.csv
File saved to /Users/andresuchitra/dev/missglam/autopo/notebook/output/emergency/9. Miss Glam Medan.csv
  - Location: MEDAN
  - Contribution: 46%
  - Rows processed: 37161
  - 'Miss Glam Padang' suppliers: 189 rows
  - Other suppliers: 11392 rows
  - No supplier data: 25580 rows
  - Saved to: /Users/andresuchitra/dev/missglam/autopo/notebook/output/complete/9. Miss Gla

,file,location,contribution_pct,total_rows,padang_suppliers,other_suppliers,no_supplier,status,output_path
0,1. Miss Glam Padang.xlsx,PADANG,100,37318,11147,537,25634,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
1,10. Miss Glam Palembang.xlsx,PALEMBANG,26,37060,209,11336,25515,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
2,11. Miss Glam Damar.xlsx,DAMAR,91,37467,6,11668,25793,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
3,12. Miss Glam Bangka.xlsx,BANGKA,28,37007,274,11221,25512,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
4,13. Miss Glam Payakumbuh.xlsx,PAYAKUMBUH,47,37100,112,11415,25573,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
5,14. Miss Glam Solok.xlsx,SOLOK,37,37085,110,11408,25567,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
6,15. Miss Glam Tembilahan.xlsx,TEMBILAHAN,27,36914,202,11200,25512,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
7,16. Miss Glam Lubuk Linggau.xlsx,LUBUK LINGGAU,26,36983,275,11167,25541,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
8,17. Miss Glam Dumai.xlsx,DUMAI,36,37020,170,11320,25530,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...
9,18. Miss Glam Kedaton.xlsx,KEDATON,18,37049,200,11284,25565,Success,/Users/andresuchitra/dev/missglam/autopo/noteb...



Sample of the last processed file:


,Brand,Kategori Brand,SKU,Nama,Toko,Stok,Daily Sales,Max. Daily Sales,Lead Time,Max. Lead Time,...,Nama Supplier,ID Brand,Nama Brand,ID Store,Nama Store,Hari Order,Min. Purchase,Trading Term,Promo Factor,Delay Factor
0,ACNAWAY,BRAND VIRAL,10400614911,ACNAWAY 3 in 1 Acne Sun Serum Sunscreen Serum ...,Miss Glam Medan,6.00,0.03,1.00,5.00,28.00,...,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480.00,ACNAWAY,19.00,Miss Glam Medan,2.00,500000.00,0.00,,
1,ACNAWAY,BRAND VIRAL,10100824612,ACNAWAY Mugwort Acne Clear Bar Soap 100gr,Miss Glam Medan,22.00,0.03,0.00,5.00,28.00,...,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480.00,ACNAWAY,19.00,Miss Glam Medan,2.00,500000.00,0.00,,
2,ACNAWAY,BRAND VIRAL,10400517459,ACNAWAY Mugwort Daily Sunscreen Only For Acne ...,Miss Glam Medan,8.00,0.34,3.00,5.00,28.00,...,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480.00,ACNAWAY,19.00,Miss Glam Medan,2.00,500000.00,0.00,,
3,ACNAWAY,BRAND VIRAL,101001107647,ACNAWAY Mugwort Gel Facial Wash Mugwort + Cent...,Miss Glam Medan,4.00,0.34,2.00,5.00,28.00,...,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480.00,ACNAWAY,19.00,Miss Glam Medan,2.00,500000.00,0.00,,
4,ACNAWAY,BRAND VIRAL,10500637717,ACNAWAY Mugwort Gel Mask Anti Pores Masker Gel...,Miss Glam Medan,0.00,0.22,0.00,5.00,28.00,...,PT. BERSAMA DISTRIVERSA INDONESIA (DC CIPUTAT),1480.00,ACNAWAY,19.00,Miss Glam Medan,2.00,500000.00,0.00,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37156,ZYWELL,DELISTING,10500322936,ZYWELL Pell Off Mask Cellendula 10g,Miss Glam Medan,0.00,0.00,0.00,0.00,0.00,...,,0.00,NaN,0.00,,0.00,0.00,0.00,,
37157,ZYWELL,DELISTING,18200200793,ZYWELL Pell Off Mask CHamomille 10g,Miss Glam Medan,0.00,0.00,0.00,0.00,0.00,...,,0.00,NaN,0.00,,0.00,0.00,0.00,,
37158,ZYWELL,DELISTING,10500300101,ZYWELL Pell Off Mask Jasmine 10g,Miss Glam Medan,0.00,0.00,0.00,0.00,0.00,...,,0.00,NaN,0.00,,0.00,0.00,0.00,,
37159,ZYWELL,DELISTING,10500322858,ZYWELL Pell Off Mask Rose 10g,Miss Glam Medan,0.00,0.00,0.00,0.00,0.00,...,,0.00,NaN,0.00,,0.00,0.00,0.00,,


In [ ]:
# Override merge_with_suppliers to prevent filling supplier data for rows with empty Brand
print("Applying empty-Brand-safe merge_with_suppliers override...")

def merge_with_suppliers(df_clean, supplier_df):
    """Merge PO data with supplier info, skipping fallback for blank Brand rows."""
    print("Merging with suppliers (override)...")

    supplier_clean = supplier_df.copy()
    supplier_clean['Nama Brand'] = supplier_clean['Nama Brand'].astype(str).str.strip()
    supplier_clean['Nama Store'] = supplier_clean['Nama Store'].astype(str).str.strip()
    supplier_clean = supplier_clean.drop_duplicates(subset=['Nama Brand', 'Nama Store'])

    df_clean = df_clean.copy()
    df_clean['Brand'] = df_clean['Brand'].astype(str).str.strip()
    df_clean['Toko'] = df_clean['Toko'].astype(str).str.strip()

    merged_df = pd.merge(
        df_clean,
        supplier_clean,
        left_on=['Brand', 'Toko'],
        right_on=['Nama Brand', 'Nama Store'],
        how='left',
        suffixes=('_clean', '_supplier')
    )

    merged_df['_brand_clean'] = merged_df['Brand'].astype(str).str.strip()
    unmatched_mask = merged_df['Nama Brand'].isna()
    fallback_mask = unmatched_mask & (merged_df['_brand_clean'] != '')

    if fallback_mask.any():
        print(
            f"Found {fallback_mask.sum()} rows without store match; attempting brand-only fallback for non-empty brands..."
        )
        unmatched_rows = merged_df[fallback_mask].copy()
        supplier_cols = [
            col for col in supplier_clean.columns if col in unmatched_rows.columns and col != 'Brand'
        ]
        unmatched_rows = unmatched_rows.drop(columns=supplier_cols, errors='ignore')
        unmatched_rows['Brand'] = unmatched_rows['_brand_clean']

        fallback_suppliers = supplier_clean.drop_duplicates(subset=['Nama Brand'])
        matched_fallback = pd.merge(
            unmatched_rows,
            fallback_suppliers,
            left_on='Brand',
            right_on='Nama Brand',
            how='left',
            suffixes=('_clean', '_supplier')
        )

        matched_initial = merged_df[~fallback_mask]
        merged_df = pd.concat([matched_initial, matched_fallback], ignore_index=True)

    merged_df = merged_df.drop(columns=['_brand_clean'], errors='ignore')

    supplier_columns = [
        'ID Supplier', 'Nama Supplier', 'ID Brand', 'ID Store',
        'Nama Store', 'Hari Order', 'Min. Purchase', 'Trading Term',
        'Promo Factor', 'Delay Factor'
    ]

    for col in supplier_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].fillna('' if merged_df[col].dtype == 'object' else 0)

    return merged_df